In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='http://www.anmm.org.mx/GMM/'
url_root= 'http://www.anmm.org.mx/GMM/'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
b = soup.find('div', class_='panel-group')

In [ ]:
box = b.findAll('div', class_='panel-body')

In [ ]:
vol = [x.find('a').get('href')for x in box]
vol=[urljoin(url_root,i) for i in vol]
vol

In [ ]:
def get_url_items(sopa, url): 
    b = soup.find('div', class_='panel-group')
    box = b.findAll('div', class_='panel-body')
    vol = [x.find('a').get('href')for x in box]
    return vol

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
  list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Gaceta Médica de México'
    a='III. Medicina y Ciencias de la Salud'
    te='Medicina'
    s_item=BeautifulSoup(r.text,'html.parser')
    try:
        titulo=tr
        content_book['Titulo Revista']=titulo
    except AttributeError:
        content_book['Titulo Revista']=None
    try:
        titulo=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    try:
        titulo=te
        content_book['Tematica']=te
    except AttributeError:
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('div', id='accordion').find('a', class_='seleccion').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    #obtener autor
    try:
        autor=s_item.find('div', class_='panel-body').get_text(strip=True)
        content_book['Resumen']=autor
    except AttributeError:
        content_book['Resumen']=None
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', class_='panel-body').find('br').find('a').get('href')
        content_book['Link Articulo']=ancla_link
    except AttributeError:
        content_book['Link Articulo']=None
    return content_book

In [ ]:
 list_scraper=list_scraper[0:90]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revist38.csv', index=False)